In [1]:
import pandas as pd
import numpy as np

from memory_profiler import profile

from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import torch
import torch.nn as nn

import time
import os
import sys

working_directory = "/home/falcetta/PINPOINT_Secret"

device = "cpu"
module_path = os.path.abspath(working_directory)
sys.path.append(module_path) 

from pycrcnn.net_builder.encoded_net_builder_ts import build_from_pytorch
from pycrcnn.crypto.crypto import encrypt_matrix, decrypt_matrix
from train_utils import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Models

In [2]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [3]:
experiment_name = "Electricity_132"
seq_length = 14
forecast_horizon = 7
model_class = "PINPOINT_2CONV"

In [4]:
model = torch.load(f"{working_directory}/Experiments/models/{experiment_name}_{forecast_horizon}_{model_class}.pt")

In [5]:
model

PINPOINT_2CONV(
  (main): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,))
    (1): Square()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (3): Square()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): Linear(in_features=64, out_features=7, bias=True)
  )
)

# Dataset

In [6]:
df = pd.read_csv(f"{working_directory}/data/Sensor_MT_132.csv", parse_dates=['Date'], index_col='Date')
df = df.iloc[:, 0]
df.index.freq = 'D'
entire_ts = df

train = entire_ts.loc[:pd.Timestamp("2014-01-01")]
validation_length = int(0.05 * len(train))
validation = entire_ts.loc[train.index[-1] + entire_ts.index.freq:train.index[-1] + validation_length * entire_ts.index.freq]
test = entire_ts.loc[validation.index[-1] + entire_ts.index.freq:]

plot_name = "Consumption"
yaxis_name = "Daily mean (kW)"

train = train.append(validation)

print(train)
print(test)

Date
2011-01-01    38.846299
2011-01-02    36.196386
2011-01-03     5.783257
2011-01-04     3.946223
2011-01-05     0.000000
                ...    
2014-02-20    10.954169
2014-02-21    10.750054
2014-02-22    28.235902
2014-02-23    39.122034
2014-02-24    11.226323
Freq: D, Name: MT_132, Length: 1151, dtype: float64
Date
2014-02-25     9.253211
2014-02-26    10.818093
2014-02-27    11.974744
2014-02-28    10.613978
2014-03-01    25.922600
                ...    
2014-12-28    28.031788
2014-12-29    10.001633
2014-12-30    11.702591
2014-12-31    18.778576
2015-01-01    45.721750
Freq: D, Name: MT_132, Length: 311, dtype: float64


# Expected outputs

In [7]:
expected_output = []

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    model.eval()

    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)

    predict = model(inputs_normalized.reshape(1, 1, seq_length))
    predict = scaler.inverse_transform(predict.cpu().detach().numpy())
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

expected_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [8]:
expected_output

Date
2014-02-25     9.324562
2014-02-26    10.174006
2014-02-27    10.206169
2014-02-28    13.620146
2014-03-01    27.399176
                ...    
2014-12-28    26.230833
2014-12-29     9.791463
2014-12-30     8.380541
2014-12-31    15.571192
2015-01-01    15.616082
Freq: D, Length: 311, dtype: float64

In [9]:
print(experiment_name)
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")

Electricity_132
MAE of model PINPOINT_2CONV, forecast horizon: 7: 2.8


## Encode the models

In [10]:
HE = Pyfhel()    
HE.contextGen(p=96155351715128, m=8192, intDigits=16, fracDigits=64) 
HE.keyGen()
HE.relinKeyGen(30, 3)

encoded_model = build_from_pytorch(HE, model.cpu().main)

# Encrypted processing

In [11]:
decrypted_output = None

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = inputs_normalized[-seq_length:].reshape(1, 1, seq_length)

    encrypted_input = encrypt_matrix(HE, inputs_normalized)

    for layer in encoded_model:
        encrypted_input = layer(encrypted_input)
    
    _min = HE.encodeFrac(scaler.min_[0])
    _scale = HE.encodeFrac(1.0 / scaler.scale_[0])
    
    encrypted_input = [list(map(lambda x: (x - _min)*_scale, encrypted_input[0]))]
    predict = decrypt_matrix(HE, encrypted_input)

    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

decrypted_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [12]:
expected_output

Date
2014-02-25     9.324562
2014-02-26    10.174006
2014-02-27    10.206169
2014-02-28    13.620146
2014-03-01    27.399176
                ...    
2014-12-28    26.230833
2014-12-29     9.791463
2014-12-30     8.380541
2014-12-31    15.571192
2015-01-01    15.616082
Freq: D, Length: 311, dtype: float64

In [13]:
decrypted_output

Date
2014-02-25     9.325462
2014-02-26    10.177539
2014-02-27    10.205378
2014-02-28    13.621875
2014-03-01    27.399855
                ...    
2014-12-28    26.229835
2014-12-29     9.792685
2014-12-30     8.382374
2014-12-31    15.571093
2015-01-01    15.613610
Freq: D, Length: 311, dtype: float64

In [14]:
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")
print(f"MAE of model {model} used on encrypted inputs, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, decrypted_output), 2)}")

MAE of model PINPOINT_2CONV, forecast horizon: 7: 2.8
MAE of model PINPOINT_2CONV used on encrypted inputs, forecast horizon: 7: 2.8


Difference between expected and obtained on encrypted data:

In [15]:
print(expected_output - decrypted_output)

Date
2014-02-25   -0.000900
2014-02-26   -0.003532
2014-02-27    0.000791
2014-02-28   -0.001729
2014-03-01   -0.000680
                ...   
2014-12-28    0.000998
2014-12-29   -0.001222
2014-12-30   -0.001833
2014-12-31    0.000099
2015-01-01    0.002472
Freq: D, Length: 311, dtype: float64
